In [7]:
# import libraries
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow import keras
from google.colab import files
import seaborn as sns
import warnings

import random
import librosa, IPython
import librosa.display as lplt
sns.set_style('whitegrid')
%matplotlib inline
seed = 12
np.random.seed(seed)
warnings.filterwarnings('ignore')

Loading the datasets

In [8]:
Train=pd.read_csv("/content/Train.csv")

In [9]:
Test=pd.read_csv("/content/Test.csv")

In [10]:
Train=Train.drop(['ID'],axis=1) #Drop the ID column in the train Dataset

In [11]:
Test=Test.drop('ID', axis=1) #Drop the ID column in the Test Dataset

In [12]:
X_val=Train[Train['country']=="Guinea"] #Splitting the Train set by countyr(guinea)

In [13]:
X_val

,country,year,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target
4,Guinea,2012,U,0.000000,0.011649,0.017560,0.017383,0.099875,0.853533,31.734661,5.081620,22.815984,0.005047,0.130475,1.461894,222.867189,192.926363,0.605328
83,Guinea,2012,R,0.000000,0.000000,0.000072,0.000469,0.010801,0.988658,9.396592,7.771570,3.645846,0.000000,0.000000,0.000000,224.742328,191.075465,0.194010
121,Guinea,2012,R,0.000000,0.000373,0.000820,0.002069,0.000508,0.996230,6.650227,11.531870,0.947606,0.000285,0.070643,0.000000,152.723675,20.987818,0.232571
154,Guinea,2012,R,0.000000,0.000000,0.000000,0.000000,0.000054,0.999946,2.444998,12.987240,2.067382,0.000000,0.675006,0.000000,326.165952,297.131871,0.290080
165,Guinea,1999,U,0.000000,0.083101,0.064724,0.062089,0.083171,0.706915,123.764385,15.443530,32.513295,0.130533,1.125885,3.366133,108.543696,87.610351,0.477963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21391,Guinea,1999,U,0.206061,0.440912,0.089261,0.026805,0.016318,0.220644,589.877513,0.983788,71.752498,2.901246,2.326802,23.741983,12.200927,14.105058,0.488366
21402,Guinea,2012,R,0.000000,0.000036,0.000146,0.000073,0.000157,0.999588,0.984063,1.275874,0.380291,0.000000,0.296059,0.000000,82.524769,47.581878,0.160343
21407,Guinea,2012,R,0.000000,0.000000,0.000000,0.000090,0.021883,0.978027,12.090547,11.889860,1.343607,0.000000,0.000000,0.000000,245.661278,216.920566,0.184565
21425,Guinea,2012,R,0.000000,0.000000,0.000000,0.000036,0.000816,0.999147,0.645471,0.837050,0.179539,0.000394,0.097780,0.000000,165.592261,136.742942,0.183879


In [14]:
X_train=Train[Train["country"] != "Guinea"] #X_train set is a set of country without guinea

In [15]:
X_val_1=X_val.drop(['Target','country','year'], axis=1) ## drop the Target country and year

In [16]:
y_val=X_val['Target'] # save the column Target into y_val

In [17]:
X_train_1=X_train.drop(['Target','country','year'], axis=1)

In [18]:
y_train=X_train["Target"]

In [15]:
# y=Train['Target']

In [16]:
# X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.15,random_state=42)

Label encoding of the country

In [17]:
# map labels to index
country_index = dict()
index_country = dict()
for i, x in enumerate(X_train_1.country.unique()):
    country_index[x] = i
    index_country[i] = x
print(country_index)
print(index_country)

{'Ethiopia': 0, 'Mozambique': 1, 'Malawi': 2, 'Cameroon': 3, 'Ghana': 4, 'Senegal': 5, 'Kenya': 6, 'Tanzania': 7, 'Mali': 8, 'Swaziland': 9, 'Rwanda': 10, 'Nigeria': 11, 'Lesotho': 12, 'Sierra Leone': 13, 'Central African Republic': 14, "Cote d'Ivoire": 15, 'Togo': 16}
{0: 'Ethiopia', 1: 'Mozambique', 2: 'Malawi', 3: 'Cameroon', 4: 'Ghana', 5: 'Senegal', 6: 'Kenya', 7: 'Tanzania', 8: 'Mali', 9: 'Swaziland', 10: 'Rwanda', 11: 'Nigeria', 12: 'Lesotho', 13: 'Sierra Leone', 14: 'Central African Republic', 15: "Cote d'Ivoire", 16: 'Togo'}


In [18]:
X_train_1.country = [country_index[l] for l in X_train_1.country]

In [19]:
# map labels to index
country_index = dict()
index_country = dict()
for i, x in enumerate(X_val_1.country.unique()):
    country_index[x] = i
    index_country[i] = x
print(country_index)
print(index_country)

{'Guinea': 0}
{0: 'Guinea'}


In [20]:
X_val_1.country = [country_index[l] for l in X_val.country]

In [21]:
# map labels to index
country_index = dict()
index_country = dict()
for i, x in enumerate(Test.country.unique()):
    country_index[x] = i
    index_country[i] = x
print(country_index)
print(index_country)

{'Democratic Republic of Congo': 0, 'Uganda': 1, 'Burkina Faso': 2, 'Zambia': 3, 'Angola': 4, 'Zimbabwe': 5, 'Benin': 6}
{0: 'Democratic Republic of Congo', 1: 'Uganda', 2: 'Burkina Faso', 3: 'Zambia', 4: 'Angola', 5: 'Zimbabwe', 6: 'Benin'}


In [19]:
Test=Test.drop(['country','year'], axis=1)

In [22]:
Test.country = [country_index[l] for l in Test.country]

one hot encoding of the urban or rural column

In [20]:
X_train_1['urban_or_rural']=pd.get_dummies(X_train_1['urban_or_rural'])

In [21]:
X_train_1

,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline
0,1,0.0,0.000000,0.000000,0.000055,0.000536,0.999408,12.146134,25.489659,0.879484,0.000000,0.000000,0.000000,278.788451,769.338378
1,1,0.0,0.000000,0.000110,0.000000,0.000018,0.999872,113.806716,64.136053,0.601427,0.000000,0.005427,0.000000,200.986978,337.135243
2,1,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.400096,0.131900,0.000000,0.003078,0.000000,642.594208,169.913773
3,1,0.0,0.000141,0.000181,0.000254,0.000228,0.999195,5.213320,25.379371,2.017136,11.293841,0.131035,0.000000,365.349451,613.591610
5,0,0.0,0.008623,0.019409,0.059886,0.082682,0.829400,203.580507,24.629433,31.235708,0.000000,0.008223,22.981970,9.803702,487.790852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.0,0.002961,0.008240,0.002313,0.008068,0.978418,44.044352,12.551978,6.302902,0.000000,0.000000,1.955632,283.861037,159.790057
21450,1,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,27.848571,0.372267,0.000000,0.000000,0.000000,295.307249,122.976960
21451,1,0.0,0.000536,0.000092,0.000018,0.000074,0.999279,0.458143,3.679934,1.712136,0.000000,0.000000,0.442422,166.405249,155.365355
21452,1,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,12.331763,0.960163,0.001899,0.061126,0.000000,568.759665,534.638628


In [22]:
X_val_1['urban_or_rural']=pd.get_dummies(X_val['urban_or_rural'])

In [23]:
Test['urban_or_rural']=pd.get_dummies(Test['urban_or_rural'])

Label encoding of the year variable

In [27]:
# map year in train to index
year_index = dict()
index_year = dict()
for i, x in enumerate(X_train_1.year.unique()):
    year_index[x] = i
    index_year[i] = x
print(year_index)
print(index_year)

{2016: 0, 2005: 1, 2009: 2, 2015: 3, 2004: 4, 2014: 5, 2010: 6, 2011: 7, 1998: 8, 2006: 9, 2008: 10, 2003: 11, 2012: 12, 2007: 13, 2013: 14, 1994: 15, 1996: 16, 1999: 17, 1995: 18}
{0: 2016, 1: 2005, 2: 2009, 3: 2015, 4: 2004, 5: 2014, 6: 2010, 7: 2011, 8: 1998, 9: 2006, 10: 2008, 11: 2003, 12: 2012, 13: 2007, 14: 2013, 15: 1994, 16: 1996, 17: 1999, 18: 1995}


In [28]:
X_train_1.year = [year_index[l] for l in X_train_1.year]

In [29]:
# map year in val to index
year_index = dict()
index_year = dict()
for i, x in enumerate(X_val_1.year.unique()):
    year_index[x] = i
    index_year[i] = x
print(year_index)
print(index_year)

{2012: 0, 1999: 1}
{0: 2012, 1: 1999}


In [30]:
X_val_1.year = [year_index[l] for l in X_val_1.year]

In [31]:
# map year in test data to index
year_index = dict()
index_year = dict()
for i, x in enumerate(Test.year.unique()):
    year_index[x] = i
    index_year[i] = x
print(year_index)
print(index_year)

{2007: 0, 2011: 1, 2010: 2, 2015: 3, 2013: 4, 2005: 5, 2014: 6, 2012: 7, 2016: 8, 2006: 9, 1999: 10, 2009: 11, 1998: 12}
{0: 2007, 1: 2011, 2: 2010, 3: 2015, 4: 2013, 5: 2005, 6: 2014, 7: 2012, 8: 2016, 9: 2006, 10: 1999, 11: 2009, 12: 1998}


In [32]:
Test.year = [year_index[l] for l in Test.year]

In [24]:
X_train_1.sample(5)

,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline
2350,0,0.030539,0.059488,0.163061,0.001399,0.055190,0.690322,244.843773,8.791062,33.493507,3.329922,1.580360,35.452853,9.209242,677.617171
3994,0,0.000000,0.004836,0.004744,0.005602,0.010730,0.974088,27.842311,46.920630,6.975733,0.000000,0.004848,3.260543,477.306873,675.711289
8288,0,0.000000,0.484165,0.000000,0.117152,0.121220,0.277464,802.736674,2.350065,81.163655,0.027856,0.237662,51.214714,386.498600,11.608316
7886,0,0.000000,0.000000,0.000000,0.000000,0.000038,0.999962,13.889355,32.974566,0.465033,1.258490,2.847156,0.000000,126.117994,497.762204
4052,0,0.023141,0.754091,0.000000,0.016665,0.014575,0.191528,563.618507,0.588059,76.135312,0.743212,0.291635,139.746702,211.935118,8.042227


Transformation of the datasets using MinMaxScaler

In [25]:
from sklearn.preprocessing import MinMaxScaler


In [26]:
def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))*20
    #dataNorm["country"]=dataset["country"]
    #dataNorm["year"]=dataset["year"]
    dataNorm["urban_or_rural"]=dataset["urban_or_rural"]
    return dataNorm

In [27]:
X_train=normalize(X_train_1)

In [28]:
X_train

,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline
0,1,0.0,0.000000,0.000000,0.002153,0.016527,19.988151,0.139510,6.367248,0.178062,0.000000,0.000000,0.000000,2.937765,8.694712
1,1,0.0,0.000000,0.003207,0.000000,0.000564,19.997435,1.307179,16.021013,0.121766,0.000000,0.001931,0.000000,2.117613,3.809437
2,1,0.0,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,1.099132,0.026705,0.000000,0.001095,0.000000,6.772857,1.919301
3,1,0.0,0.003204,0.005294,0.009868,0.007035,19.983896,0.059880,6.339698,0.408393,2.277810,0.046630,0.000000,3.850256,6.934276
5,0,0.0,0.196631,0.566678,2.323247,2.547343,16.585074,2.338317,6.152366,6.324036,0.000000,0.002926,1.200314,0.102237,5.512326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.0,0.067512,0.240594,0.089736,0.248571,19.567979,0.505892,3.135450,1.276097,0.000000,0.000000,0.102140,2.991238,1.804871
21450,1,0.0,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,6.956498,0.075370,0.000000,0.000000,0.000000,3.111899,1.388765
21451,1,0.0,0.012231,0.002700,0.000718,0.002279,19.985560,0.005262,0.919238,0.346642,0.000000,0.000000,0.023107,1.753066,1.754857
21452,1,0.0,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,3.080441,0.194396,0.000383,0.021752,0.000000,5.994523,6.041856


In [29]:
X_val=normalize(X_val_1)

In [30]:
Test=normalize(Test)

In [40]:
# # define the model
# def baseline():
#     # create model
#     model = Sequential()
#     # add one fully connected layer
#     model.add(Dense(13, input_dim=X_train.shape[1], activation='relu'))
#     # add a fully connected layer for the output
#     model.add(Dense(1))
#     # Compile model
#     model.compile(loss='mean_squared_error', optimizer='adam',metrics=[metrics.mse])
    
#     return model

In [41]:
# estimator = KerasRegressor(build_fn=baseline, epochs=20, batch_size=5, verbose=1)

In [42]:
# baseline().summary()

In [43]:
# history = estimator.fit(X_train, y_train)

In [44]:
# prediction = estimator.predict(X_val)

In [45]:
# from sklearn.metrics import mean_squared_error

In [46]:
# mean_squared_error(y_val, prediction)

In [47]:
# prediction = estimator.predict(Test)

In [48]:
# submission=pd.read_csv("/content/SampleSubmission.csv")
# submission.head()

In [49]:
# submission['Target']=prediction = estimator.predict(Test)

In [50]:
# submission.head(5)

In [51]:
# submission.to_csv("Zindi2.csv",index=False)

Model building

In [54]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=False)



[21:21:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [55]:
# make predictions
predictions = my_model.predict(X_val)




In [56]:
predictions

array([0.534668  , 0.22236732, 0.24015382, 0.19637096, 0.59565955,
       0.40890968, 0.23533887, 0.16450664, 0.16265827, 0.17457393,
       0.4454319 , 0.17482916, 0.39138097, 0.75501156, 0.7316047 ,
       0.5774448 , 0.1335795 , 0.7195407 , 0.52093625, 0.58953565,
       0.19118154, 0.24803138, 0.17991251, 0.74634004, 0.17748421,
       0.1329984 , 0.45213997, 0.38982502, 0.23969361, 0.5599706 ,
       0.44441375, 0.73679674, 0.3910555 , 0.7076104 , 0.19427264,
       0.24358886, 0.38421032, 0.5476099 , 0.16634664, 0.14844128,
       0.7355213 , 0.18227544, 0.22510901, 0.16746   , 0.18096253,
       0.21875018, 0.21337157, 0.21370563, 0.14119902, 0.31403637,
       0.22415239, 0.34197325, 0.6169809 , 0.1544793 , 0.13094962,
       0.18243966, 0.75501156, 0.223311  , 0.59866476, 0.14368   ,
       0.16579664, 0.24022964, 0.6250006 , 0.47970423, 0.7453054 ,
       0.12517938, 0.22466367, 0.21928632, 0.19525072, 0.765519  ,
       0.5474045 , 0.22881237, 0.3276687 , 0.18008253, 0.75733

In [58]:
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_val)))

Mean Absolute Error : 0.08552723575216409


In [35]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_val, y_val)], verbose=False)

[21:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [36]:

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_val, y_val)], verbose=False)



[21:10:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [37]:
# make predictions
predictions = my_model.predict(X_val)

In [38]:
predictions

array([0.5509291 , 0.24693146, 0.25871956, 0.23781225, 0.6034306 ,
       0.44191223, 0.25784612, 0.22206378, 0.21805951, 0.2242209 ,
       0.44545642, 0.20968431, 0.43085644, 0.6464954 , 0.6449618 ,
       0.5580857 , 0.19531268, 0.6421013 , 0.5137094 , 0.6034306 ,
       0.23844612, 0.2566892 , 0.22203746, 0.6421013 , 0.22016597,
       0.18888113, 0.45015594, 0.4714859 , 0.25784612, 0.5530993 ,
       0.45013762, 0.64879394, 0.43588412, 0.64879394, 0.23416683,
       0.25568897, 0.43588412, 0.5393298 , 0.22700283, 0.18250737,
       0.6599381 , 0.22016597, 0.261247  , 0.21123862, 0.23006251,
       0.24477434, 0.24989411, 0.23307532, 0.21812686, 0.3979944 ,
       0.24693146, 0.35246086, 0.6003949 , 0.21689653, 0.17976525,
       0.23006251, 0.6464954 , 0.24693146, 0.59037113, 0.18798119,
       0.2242209 , 0.25589156, 0.6004638 , 0.48993927, 0.6421013 ,
       0.16800955, 0.24070102, 0.2496076 , 0.24079785, 0.6599381 ,
       0.5393298 , 0.24740756, 0.3646083 , 0.236289  , 0.64649

In [39]:
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_val)))

Mean Absolute Error : 0.07529822214349709


In [ ]:
submission['Target']=my_model.predict(Test)

In [ ]:
submission.head(5)

,ID,Target
0,ID_AAcismbB,0.249722
1,ID_AAeBMsji,0.254857
2,ID_AAjFMjzy,0.579152
3,ID_AAmMOEEC,0.280792
4,ID_ABguzDxp,0.393514


In [ ]:
submission.to_csv("Zindi3.csv",index=False)

Model with regularizer (ElasticNet)

ElasticNet Model

In [40]:
import sklearn

In [41]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression , ElasticNet

In [42]:
from sklearn.model_selection import GridSearchCV , KFold

In [43]:
cv = KFold(n_splits=10)

In [59]:
Ela = ElasticNet(alpha=0.5 , l1_ratio=0.5)
Ela.fit(X_train, y_train)


ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [45]:
# define grid
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = np.arange(0, 0.1, 0.001)

In [46]:
# define search
search = GridSearchCV(Ela, grid, scoring='neg_mean_squared_error', cv=cv , n_jobs=-1)
#sorted(sklearn.metrics.SCORERS.keys())

In [47]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [60]:
# perform the search
results = search.fit(X_train, y_train)

In [ ]:
# summarize
mse = -1*results.best_score_
print('MSE: %.3f' % mse)
print('Config: %s' % results.best_params_)

In [64]:
Ela_gs = ElasticNet(**results.best_params_)

In [65]:
Ela_gs.fit(X_train, y_train)

ElasticNet(alpha=0.0001, copy_X=True, fit_intercept=True, l1_ratio=0.14,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [66]:
y_h_Ela_train = Ela_gs.predict(X_train)
y_h_Ela_test = Ela_gs.predict(X_val)

In [70]:
from sklearn.metrics import mean_squared_error

In [71]:
print('Train Error Elastic Net Model', mean_squared_error(y_train , Ela_gs.predict(X_train).reshape(-1,1)))
print('Test Error Elastic Net  Model', mean_squared_error(y_val , Ela_gs.predict(X_val).reshape(-1,1)))

Train Error Elastic Net Model 0.014369239791936467
Test Error Elastic Net  Model 0.026539100680753453


In [74]:
submission=pd.read_csv("/content/SampleSubmission.csv")

In [75]:
submission['Target']=Ela_gs.predict(Test)

In [76]:
submission.head(5)

,ID,Target
0,ID_AAcismbB,0.022707
1,ID_AAeBMsji,0.257180
2,ID_AAjFMjzy,0.577656
3,ID_AAmMOEEC,0.379615
4,ID_ABguzDxp,0.210527


In [77]:
submission.to_csv("Zindi4.csv",index=False)

Catboost